# LightGBM

LightGBM, XGBoost'un eğitim süresi performansını arttırmaya yöntelik geliştirilen bir diğer GBM türüdür.

Level-wise büyüme stratejisi yerine Leaf-wise büyüme stratejisi ile daha hızlıdır.

XGBoost geniş kapsamlı bir ilk arama yaparken LightGBM derinlemesine ilk arama yapmaktadır.

## LightGBM Uygulama

In [2]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve

warnings.simplefilter(action='ignore', category=Warning)

from xgboost import XGBClassifier
# !pip install lightgbm
from lightgbm import LGBMClassifier
# !pip install catboost
# from catboost import CatBoostClassifier

In [3]:
df = pd.read_csv("datasets/diabetes.csv")

In [4]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

### LightGBM 

In [5]:
lgbm_model = LGBMClassifier(random_state=17)

In [6]:
lgbm_model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': 17,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

Hiperparametre optimizasyonu yapmadan önceki hatalarımız:

In [7]:
cv_results = cross_validate(lgbm_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
print("Accuracy: ", cv_results["test_accuracy"].mean())
print("F1: ", cv_results["test_f1"].mean())
print("ROC-AUC: ", cv_results["test_roc_auc"].mean())

Accuracy:  0.7474492827434004
F1:  0.624110522144179
ROC-AUC:  0.7990293501048218


Şimdi optimizasyon işlemi için hiperparametrelerin ve değerlerinin tutulduğu bir sözlük yapısı oluşturalım.

In [8]:
lgbm_params = {"learning_rate": [0.01, 0.1, 0.001],
              "n_estimators": [100, 300, 500, 1000],
              "colsample_bytree": [0.5, 0.7, 1]}

In [9]:
lgbm_best_grid = GridSearchCV(lgbm_model, lgbm_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.1min finished


In [10]:
lgbm_best_grid.best_params_

{'colsample_bytree': 1, 'learning_rate': 0.01, 'n_estimators': 300}

In [11]:
lgbm_final = lgbm_model.set_params(**lgbm_best_grid.best_params_, random_state=17).fit(X, y)

In [12]:
cv_results = cross_validate(lgbm_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
print("Accuracy: ", cv_results["test_accuracy"].mean())
print("F1: ", cv_results["test_f1"].mean())
print("ROC-AUC: ", cv_results["test_roc_auc"].mean())

Accuracy:  0.7643578643578645
F1:  0.6372062920577772
ROC-AUC:  0.8147491264849755


LightGBM için en önemli parametre n_estimators (iterasyon) sayısıdır. Diğer hiperparametrelerin kombinasyonlarını belirledikten sonra tek başına n_estimators parametresini aramamız gerekir (10000lere kadar çıkabilir). Dolayısıyla öncelikle belirlediğimiz parametreleri baştan modele girip daha sonrasında yukarıdaki işlemi sadece n_estimators için yapmamız gerekli. 

In [13]:
lgbm_model = LGBMClassifier(random_state=17, colsample_bytree=1, learning_rate=0.01)

In [16]:
lgbm_params = {"n_estimators": [300, 500, 750, 1000, 5000, 10000]}

In [17]:
lgbm_best_grid = GridSearchCV(lgbm_model, lgbm_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   57.8s finished


In [18]:
lgbm_best_grid.best_params_

{'n_estimators': 300}

n_estimators için en verimli değerin değişmemesi veri setinin küçüklüğüyle alakalı olabilir. Fakat yine de eğer LightGBM kullanılıyorsa, 5000, 10000, 15000 gibi değerlerin mutlaka denenmesi gerekir. Bu değerlerle ilişkili bir şey çıkmıyorsa her halükarda denendi ve en iyisi buymuş yorumu yapılır.